In [8]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from os import path, listdir, mkdir
from scipy.stats import iqr
sns.set(style = 'whitegrid')
markers = {"UP": "o", "DOWN": "X"}

This function selects DE proteins with the method chosen (see 'method'). Based on fold change and fdr values of these proteins following metrics are calculated:

$$ E = \sqrt{(\overline{log_{2}FC})^{2} + (\overline{-log_{10}FDR})^{2}} $$
$$ E_{m} = \sqrt{(\overline{log_{2}FC} - T_{FC})^{2} + (\overline{-log_{10}FDR} - T_{FDR})^{2}} $$
$$ \pi_{1} = \sum_{i = 1} ^{n} | log_{2}FC_{i} \cdot (-log_{10}FDR_{i})| $$
$$ \pi_{2} = log_{10} (\prod_{i = 1} ^{n} | log_{2}FC_{i} \cdot (-log_{10}FDR_{i})| ) $$
$ T_{FC}$ and  $T_{FDR}$ correspond to fold change threshold value and fdr threshold value respectively

**Input**:

Tab-separated files with quantitative analysis results are required for the script. Input files should contain 'log2(fold_change)' and '-log10(fdr_BH)' columns.
You can provide several files from different datasets (see 'path_to_data'). Output plots will contain metric values for all the files provided. Value corresponding to files from one dataset will be designated with the marker of the same color.
Names of the files should fit the following structure: *begin_pattern|dataname|end_pattern* and *dataname* should be unique.

**Output**:

Tab-separated file containing metric values for all the files provided.
Tab-separated file containing paths to all the files provided, dataset name and dataname.
Metric scatterplots containing metric value for all the files provided. 

Parameters:

path_to_data - dictionary containing names of datasets as keys and path to directory with quantitative analysis results
    
dataname_to_plot - dictionary containing draft data names from filenames as keys and final datanames displayed
    in the plot as values; default None
    
begin_pattern - string, corresponding to the beginning of the all filenames

end_pattern - string, corresponding to the end of all filenames

method - how the thresholds for DE proteins should be chosen :
    'static' - thresholds for the fdr and fold change are filxed and the same for all the files (see 'fold change' and
        'alpha') 
    'semi-dynamic' - thresholds for fold change are fixed (see 'fold change'), threshold for fdr is Q3 + 1.5 IQR
    'dynamic' - upper and lower fold change thesholds are Q3+1.5 IQR and Q1-1.5 IQR respectively, 
        threshold for fdr is Q3+1.5 IQR
                
reg_type - what proteins should be used for calculation:
    'UP' - only positively regulated proteins that passed the thresholds
    'DOWN' - only negatively regulated proteins that passed the thresholds
    'DE' - all the proteins that passed the thresholds; metrics is calculated across all proteins
    'UP+DOWN' - all the proteins that passed the thresholds; separate metrics for positively and negatively regulated
        proteins is calculated
    
fold_change - threshold value for fold change; used if method is 'static' or 'semi-dynamic'; default 2

alpha - threshold value for fdr; used if method is 'static'; default 0.01

output_dir - output path

add_name - additional discription to the output filename; default None

In [59]:
def metrics(path_to_data, begin_pattern, end_pattern, method, reg_type, output_dir, 
            dataname_to_plot = None, fold_change = 2, alpha = 0.01, add_name = None):
    
    if method not in ['semi-dynamic', 'static', 'dynamic']:
        print('Error: check method')
        return
    elif reg_type not in ['DOWN', 'DE', 'UP+DOWN', 'UP']:
        print('Error: check reg_type')
        return
    for name in path_to_data.keys():
        for d in path_to_data[name]:
            if path.exists(d) == False:
                print('Error: {} not exist'.format(d))
                return
    
    sample_df = pd.DataFrame(columns = ('path to file', 'dataset', 'dataname'))
    for d in path_to_data.keys():
        for file in path_to_data[d]:
            dataname = file.split(begin_pattern)[1].split(end_pattern)[0]
            dataset = d
            tmp = pd.DataFrame.from_dict({'dataname' : [dataname_to_plot[dataname]],
                                                   'dataset' : [dataset],
                                                   'path to file':[file]})
            sample_df = pd.concat([sample_df, tmp], ignore_index = True)
    
    if add_name:
        output_filename = 'metrics_sample_file_{}.tsv'.format(add_name)
    else:
        output_filename = 'metrics_sample_file.tsv'
    sample_df.to_csv(path.join(output_dir, output_filename), sep = '\t', index = 'dataname')
    
    metrics = pd.DataFrame(columns = ('pi1', 'b2', 'pi2', 'dataname', 'dataset'))

    order = sample_df['dataset'].unique()
    sns.set_palette('hls', len(order))
    cm = 1/2.54
    figsize = (20*cm, sample_df.shape[0]*cm)

    for i in sample_df.index:
        df = pd.read_csv(sample_df.loc[i, 'path to file'], sep = '\t')
        df = df[['log2(fold_change)', '-log10(fdr_BH)']]
        dataset = sample_df.loc[i, 'dataset']
        dataname = sample_df.loc[i, 'dataname']

        if method == 'static':
            t_h = -np.log10(alpha)
            up_fold = np.log2(fold_change)
            down_fold = np.log2(1/fold_change)

        elif method == 'semi-dynamic':
            t_h = np.quantile(df['-log10(fdr_BH)'], 0.75) + 1.5*iqr(df['-log10(fdr_BH)'])
            up_fold = np.log2(fold_change)
            down_fold = np.log2(1/fold_change)

        else:  
            t_h = np.quantile(df['-log10(fdr_BH)'], 0.75) + 1.5*iqr(df['-log10(fdr_BH)'])
            up_fold = np.quantile(df['log2(fold_change)'], 0.75) + 1.5*iqr(df['log2(fold_change)'])
            down_fold = np.quantile(df['log2(fold_change)'], 0.25) - 1.5*iqr(df['log2(fold_change)'])

        if reg_type == 'UP':
            de_df = df[(df['-log10(fdr_BH)'] > t_h) & (df['log2(fold_change)'] >= up_fold)]
            mean_fc = de_df['log2(fold_change)'].mean()
            mean_fdr = de_df['-log10(fdr_BH)'].mean()
            e_dist = np.sqrt(mean_fc**2 + mean_fdr**2)
            e_dist_mod = np.sqrt((mean_fc - up_fold)**2 + (mean_fdr - t_h)**2)
            
        elif reg_type == 'DOWN':
            de_df = df[(df['-log10(fdr_BH)'] > t_h) & (df['log2(fold_change)'] < down_fold)]
            mean_fc = de_df['log2(fold_change)'].mean()
            mean_fdr = de_df['-log10(fdr_BH)'].mean()
            e_dist = np.sqrt(mean_fc**2 + mean_fdr**2)
            e_dist_mod = np.sqrt((mean_fc - down_fold)**2 + (mean_fdr - t_h)**2)
            
        elif reg_type == 'UP+DOWN':
            de_df = df[(df['-log10(fdr_BH)'] > t_h) & ((df['log2(fold_change)'] < down_fold)|(df['log2(fold_change)'] >= up_fold))]
            up_df = de_df[de_df['log2(fold_change)'] > 0]
            down_df = de_df[de_df['log2(fold_change)'] < 0]
            
            mean_fc_up = up_df['log2(fold_change)'].mean()
            mean_fc_down = down_df['log2(fold_change)'].mean()
            mean_fdr_up = up_df['-log10(fdr_BH)'].mean()
            mean_fdr_down = down_df['-log10(fdr_BH)'].mean()
            
            e_dist_up = np.sqrt(mean_fc_up**2 + mean_fdr_up**2)
            e_dist_down = np.sqrt(mean_fc_down**2 + mean_fdr_down**2)
            e_dist_mod_up = np.sqrt((mean_fc_up - up_fold)**2 + (mean_fdr_up - t_h)**2)
            e_dist_mod_down = np.sqrt((mean_fc_down - down_fold)**2 + (mean_fdr_down - t_h)**2)
            
        elif reg_type == 'DE':
            de_df = df[(df['-log10(fdr_BH)'] > t_h) & ((df['log2(fold_change)'] < down_fold)|(df['log2(fold_change)'] >= up_fold))]


        if reg_type == 'UP+DOWN':
            
            tmp_up = [np.abs(up_df.loc[i, 'log2(fold_change)'] * up_df.loc[i, '-log10(fdr_BH)']) for i in up_df.index]
            tmp_down = [np.abs(down_df.loc[i, 'log2(fold_change)'] * down_df.loc[i, '-log10(fdr_BH)']) for i in down_df.index]

            metrics = pd.concat([metrics, pd.DataFrame.from_dict({'E' : [e_dist_up, e_dist_down],
                                                                 'Em' : [e_dist_mod_up, e_dist_mod_down],
                                                                 'pi1' : [np.sum(tmp_up), np.sum(tmp_down)],
                                                                 'b2' : [np.prod(tmp_up), np.prod(tmp_down)],
                                                                 'reg_type' : ['UP', 'DOWN'],
                                                                 'dataset' : [dataset, dataset],
                                                                 'dataname' : [dataname, dataname]})], 
                                axis = 0, ignore_index = True)
        elif reg_type == 'DE':
            tmp = [np.abs(de_df.loc[i, 'log2(fold_change)'] * de_df.loc[i, '-log10(fdr_BH)']) for i in de_df.index]
            metrics = pd.concat([metrics, pd.DataFrame.from_dict({'pi1' : [np.sum(tmp)],
                                                                 'b2' : [np.prod(tmp)],
                                                                 'dataset' : [dataset],
                                                                 'dataname' : [dataname]})], 
                                axis = 0, ignore_index = True)
                                 
        else:
            tmp = [np.abs(de_df.loc[i, 'log2(fold_change)'] * de_df.loc[i, '-log10(fdr_BH)']) for i in de_df.index]
            metrics = pd.concat([metrics, pd.DataFrame.from_dict({'E' : [e_dist],
                                                                 'Em' : [e_dist_mod],
                                                                 'pi1' : [np.sum(tmp)],
                                                                 'b2' : [np.prod(tmp)],
                                                                 'dataset' : [dataset],
                                                                 'dataname' : [dataname]})], 
                                axis = 0, ignore_index = True)

    metrics['pi2'] = metrics['b2'].apply(lambda x: np.log10(x))

    if path.exists(output_dir) == False:
        mkdir(output_dir)
    if add_name:
        output_filename = 'metrics_{}_{}_{}.tsv'.format(method, reg_type, add_name)
    else:
        output_filename = 'metrics_{}_{}.tsv'.format(method, reg_type)
    metrics[['dataname', 'dataset', 'reg_type', 'E', 'Em', 'pi1', 'pi2']].to_csv(path.join(output_dir, output_filename),
                                                                                 sep = '\t', index = 'dataname')

    if reg_type == 'DE':
        metric = ['pi1', 'pi2']
        metric_name = ['$ \pi_{1} = \sum_{i = 1}^{n} | \  log2FC_{i} \cdot log10FDR_{i} \ | $',
                '$ \pi_{2} = log_{10} ( \prod_{i = 1}^{n} \ | \  log2FC_{i} \cdot log10FDR_{i} \ | ) $']
    else:
        metric = ['E', 'Em', 'pi1', 'pi2']
        metric_name = ['Euclidean distance', 'Modified euclidean distance', 
               '$ \pi_{1} = \sum_{i = 1}^{n} | \  log2FC_{i} \cdot log10FDR_{i} \ | $',
                '$ \pi_{2} = log_{10} ( \prod_{i = 1}^{n} \ | \  log2FC_{i} \cdot log10FDR_{i} \ | ) $']
    
    
    for m, n in zip(metric, metric_name):
        f, ax = plt.subplots(figsize = figsize)
        
        if reg_type == 'UP+DOWN':
            
            tmp_df = metrics[['dataname', m, 'reg_type', 'dataset']]
            tmp_df = tmp_df[tmp_df['reg_type'] == 'UP']
            y_order = tmp_df.sort_values(by = m).dataname.values
            metrics['dataname_cat'] = pd.Categorical(metrics['dataname'], y_order)
            
            sns.scatterplot(x = m, y = 'dataname', hue = 'dataset', hue_order = order,
                    data = metrics.sort_values(by=['dataname_cat'], ascending = False), 
                    style = 'reg_type', markers = markers,  ax = ax, s = 100) 
        else:    
            sns.scatterplot(x = m, y = 'dataname', hue = 'dataset',
                        hue_order = order,
                        data = metrics.sort_values(by = m, ascending = False), marker = 'o',  ax = ax, s = 100) 

        ax.tick_params(axis = 'both', labelsize = 10)
        ax.set_ylabel(None)
        ax.set_xlabel(n, fontsize = 12)
        ax.set_title('{} thresholds'.format(method), fontsize = 10)
        f.tight_layout()
        plt.legend(fontsize = 10)
        if add_name:
            output_figname = 'metric_{}_{}_{}_{}.png'.format(m, method, reg_type, add_name)
        else:
            output_figname = 'metric_{}_{}_{}.png'.format(m, method, reg_type)
        plt.savefig(path.join(output_dir, output_figname), dpi = 300)
        plt.close()

# Example usage

In [25]:
path_to_data = {'DBTRG&HOS IFNa' : ['example/input_go_terms_analysis/NSAF_t-test_a172_dbtrg_A172.tsv',
                                    'example/input_go_terms_analysis/NSAF_t-test_a172_dbtrg_DBTRG.tsv'],
                'GBM2017 IFNa' : ['example/input_go_terms_analysis/NSAF_t-test_glioblastoma_2017_infa_b.tsv', 
                                  'example/input_go_terms_analysis/NSAF_t-test_glioblastoma_2017_infa_i.tsv'],
                'GBM2017 IFNb' : ['example/input_go_terms_analysis/NSAF_t-test_glioblastoma_2017_infb_b.tsv', 
                                  'example/input_go_terms_analysis/NSAF_t-test_glioblastoma_2017_infb_i.tsv'],
                'GBM2019 IFNa' : ['example/input_go_terms_analysis/NSAF_t-test_glioblastoma_2019_3821.tsv',
                                  'example/input_go_terms_analysis/NSAF_t-test_glioblastoma_2019_4114.tsv',
                                  'example/input_go_terms_analysis/NSAF_t-test_glioblastoma_2019_5522.tsv',
                                  'example/input_go_terms_analysis/NSAF_t-test_glioblastoma_2019_6067.tsv',
                                  'example/input_go_terms_analysis/NSAF_t-test_glioblastoma_2019_AN.tsv'],
                'MRC5 IFNa': ['example/input_go_terms_analysis/NSAF_t-test_time_infa_24.tsv',
                              'example/input_go_terms_analysis/NSAF_t-test_time_infa_48.tsv',
                              'example/input_go_terms_analysis/NSAF_t-test_time_infa_4.tsv'],
                'MRC5 IFNg' : ['example/input_go_terms_analysis/NSAF_t-test_time_infg_24.tsv',
                              'example/input_go_terms_analysis/NSAF_t-test_time_infg_48.tsv',
                              'example/input_go_terms_analysis/NSAF_t-test_time_infg_4.tsv']}

dataname_to_plot = {'time_infg_48':'MRC5 IFNg 500 48h',
                   'time_infg_24' : 'MRC5 IFNg 500 24h',
                   'glioblastoma_2017_infa_b' : 'GBM2017b IFNa-2b 100 24h',
                   'a172_dbtrg_DBTRG' : 'DBTRG IFNa-2b 1000 24h',
                   'a172_dbtrg_A172' : 'HOS IFNa-2b 1000 24h',
                   'time_infg_4' : 'MRC5 IFNg 500 4h',
                   'glioblastoma_2017_infa_i' : 'GBM2017i IFNa-2b 100 24h',
                   'glioblastoma_2019_5522' : 'GBM5522 IFNa-2b 100 24h',
                   'time_infa_4' : 'MRC5 IFNa-2b 500 4h',
                   'glioblastoma_2019_AN' : 'AN IFNa-2b 100 24h',
                   'glioblastoma_2017_infb_i' : 'GBM2017i IFNb-2b 1000 24h',
                   'glioblastoma_2017_infb_b' : 'GBM2017b IFNb-2b 1000 24h',
                    'time_infa_24' : 'MRC5 IFNa-2b 500 24h',
                    'glioblastoma_2019_6067' : 'GBM6067 IFNa-2b 100 24h',
                    'time_infa_48' : 'MRC5 IFNa-2b 500 48h',
                    'glioblastoma_2019_3821' : 'GBM3821 IFNa-2b 100 24h',
                    'glioblastoma_2019_4114' : 'GBM4114 IFNa-2b 100 24h'}

begin_pattern = 'NSAF_t-test_'
end_pattern = '.tsv'


method = 'static' #'static', 'semi-dynamic', 'dynamic'
reg_type = 'UP+DOWN' #'UP', 'DOWN', 'DE' 'UP+DOWN', 
fold_change = 2
alpha = 0.01
output_dir = 'example/output_metrics'
add_name = ''

In [60]:
metrics(path_to_data, 'NSAF_t-test_', '.tsv', method = 'dynamic', reg_type = 'UP+DOWN', output_dir = output_dir,
       dataname_to_plot = dataname_to_plot, add_name = add_name)